# Sklearn 机器学习介绍

# 基本概念

在深入scikit-learn之前，让我们先快速了解一些机器学习的基本概念。

计算机程序随着经验的积累，能够实现性能的提高。对于某一类任务T及其性能度量P，若一个计算机程序在T上以P衡量的性能随着经验E而自我完善，那么就称这个计算机程序在从经验E学习。

### 监督学习 vs 无监督学习

- **监督学习**：这种类型的学习涉及到使用带有标签的数据。目标是训练一个模型，使其能够预测或分类新的未标记数据。
```
有一组房屋数据，包括房屋的大小、位置、房间数量和价格（价格是标签）。我们训练模型根据大小、位置和房间数量来预测房屋价格。
```
- **无监督学习**：与监督学习不同，无监督学习使用的是没有标签的数据。这里的目标是发现数据中的模式或结构。
```
拥有一组顾客数据但没有任何标签。通过无监督学习，我们可以识别出不同的顾客群体，根据他们的购买行为或其他特征将他们分类
```
- 本质区别：数据有没有标签

# 案例分析:鸢尾花数据集
## 数据预处理

包载入

In [ ]:
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np


首先看一下特征的样子：萼片长宽，花瓣长宽

In [ ]:
# 加载鸢尾花数据集
iris = datasets.load_iris()
X = iris.data  # 获取特征数据
y = iris.target  # 获取标签数据
# 将数据集转换为DataFrame，便于处理
iris_df = pd.DataFrame(X, columns=iris.feature_names)

# 查看前几行数据
print(iris_df.head())



   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0                5.1               3.5                1.4               0.2
1                4.9               3.0                1.4               0.2
2                4.7               3.2                1.3               0.2
3                4.6               3.1                1.5               0.2
4                5.0               3.6                1.4               0.2


Y代表每种鸢尾花的种类：Setosa、Versicolour 和 Virginica

In [ ]:
iris_df_target = pd.DataFrame(y)

# 查看数据分布
print(iris_df_target.value_counts())

0
0    50
1    50
2    50
Name: count, dtype: int64


三种类型数目相同： 平衡的分类问题
### 预处理第一步：标准化

In [ ]:
# 特征缩放 - 标准化
# 标准化是特征工程中常见的一种方法，它通过减去平均值并除以标准差来调整特征，使其具有标准正态分布的特性
#这里使用的z-score标准化

scaler = StandardScaler()
iris_scaled = scaler.fit_transform(iris_df)

# 将缩放后的数据转换回DataFrame
iris_scaled_df = pd.DataFrame(iris_scaled, columns=iris.feature_names)

# 查看标准化后的数据
print(iris_scaled_df.head())

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
0          -0.900681          1.019004          -1.340227         -1.315444
1          -1.143017         -0.131979          -1.340227         -1.315444
2          -1.385353          0.328414          -1.397064         -1.315444
3          -1.506521          0.098217          -1.283389         -1.315444
4          -1.021849          1.249201          -1.340227         -1.315444


标准化的作用：
1. 统一特征尺度
2. 降低异常值的影响
3. 提高算法的性能（梯度下降算法）

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# 使用随机森林分类器评估特征重要性
# 特征缩放 - 标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

clf = RandomForestClassifier()
clf.fit(X_scaled, y)

# 获取特征重要性并打印
feature_importances = clf.feature_importances_
print("Feature Importances:", feature_importances)



Feature Importances: [0.10654092 0.0351585  0.38764395 0.47065663]


创建新的特征：示例
叶片面积


In [ ]:
# 创建新特征
# 例如，我们可以通过现有特征的某种组合来创建新特征
# 这里，我们将萼片长度和宽度的乘积作为一个新特征
sepal_area = X[:, 0] * X[:, 1]
sepal_area = sepal_area.reshape(-1, 1)#变成二维数组，方便添加  [n_samples] ---> [n_samples, 1]

# 将新特征添加到特征矩阵中
X_new = np.hstack((X_scaled, sepal_area))

# 使用新的特征集训练模型
clf.fit(X_new, y)

# 评估新特征集上的特征重要性
new_feature_importances = clf.feature_importances_
print("New Feature Importances:", new_feature_importances)

New Feature Importances: [0.10519925 0.04291426 0.40367885 0.40910176 0.03910587]


我们后续仍使用X来做机器学习，而不使用X_new

In [ ]:
#示例
from sklearn.preprocessing import PolynomialFeatures
import numpy as np

# 示例数据
X = np.array([[a, b] for a, b in zip(range(1, 4), range(4, 7))])
print(X)
# 实例化 PolynomialFeatures 对象，degree=2 表示最多到二次项
poly = PolynomialFeatures(degree=2, include_bias=False)

# 转换数据
X_poly = poly.fit_transform(X)

# 打印转换后的数据
print(X_poly)


[[1 4]
 [2 5]
 [3 6]]
[[ 1.  4.  1.  4. 16.]
 [ 2.  5.  4. 10. 25.]
 [ 3.  6.  9. 18. 36.]]


### 训练集和测试集的划分
训练集：模型用来更新参数的数据集
测试集：训练过程中没有见过的新数据，体现模型泛化能力

In [ ]:
#方法一：直接随机划分（按照比例）
from sklearn.model_selection import train_test_split

# 将数据分为训练集和测试集（例如，测试集占 20%）
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
#方法二：分层抽样
from sklearn.model_selection import train_test_split

# 使用分层抽样进行划分，保持y的分布(训练集：120个样本（每个类别40个）测试集：30个样本（每个类别10个）)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)


#### 交叉验证

K折交叉验证：
通常，数据集被划分为“K”个大小相等的子集。每次留出一个子集作为验证集（用于测试模型），其余的“K-1”个子集用作训练集。这个过程重复进行“K”次，每次选择不同的子集作为验证集。  

最终性能评估是所有“K”次测试结果的平均值。

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# 使用交叉验证
clf = RandomForestClassifier(random_state=42)
scores = cross_val_score(clf, X_scaled, y, cv=5)
print("Accuracy scores for each fold:", scores)


Accuracy scores for each fold: [0.96666667 0.96666667 0.93333333 0.96666667 1.        ]


### 建模

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from scipy import interpolate
from itertools import cycle


In [ ]:
# 选择多个模型进行比较
models = {
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "Support Vector Machine": SVC(probability=True)
}

# 交叉验证
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    print(f"{name} Accuracy: {np.mean(scores):.2f} (+/- {np.std(scores)*2:.2f})")


Random Forest Accuracy: 0.95 (+/- 0.03)
Logistic Regression Accuracy: 0.96 (+/- 0.05)
Support Vector Machine Accuracy: 0.97 (+/- 0.06)


In [ ]:
# 训练模型
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 预测
y_pred = model.predict(X_test)

# 计算混淆矩阵
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# 打印分类报告
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Confusion Matrix:
 [[10  0  0]
 [ 0  9  1]
 [ 0  1  9]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.90      0.90      0.90        10
           2       0.90      0.90      0.90        10

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



- 混淆矩阵：  
第一行 [10 0 0] 表示有10个样本真实属于类别0，模型也全部正确预测为类别0。  
第二行 [0 9 1] 表示有10个样本真实属于类别1，模型正确预测了其中9个，但有1个错误地预测为类别2。  
第三行 [0 1 9] 表示有10个样本真实属于类别2，模型正确预测了其中9个，但是有1个错误地预测为类别1。  

### 分类别的评估  
精确度（Precision）：这告诉我们，当模型预测一个样本属于某个类别时，它有多准确。例如，类别1的精确度是1.00，意味着当模型预测任何样本为类别1时，都是正确的。  

召回率（Recall）：这表示模型找到了一个类别中的多少真实样本。例如，类别1的召回率是0.90，说明模型找到了类别1中90%的真实样本。  

F1分数（F1 Score）：这是精确度和召回率的调和平均值，用于衡量模型的整体性能。例如，类别2的F1分数是0.90，是一个相对较高的分数，表明该类别的性能很好。  



支持（Support）：这是每个类别中的样本数量。在我们的例子中，每个类别都有10个样本。  

准确率（Accuracy）：这是模型正确预测的样本数占总样本数的比例。在这个例子中，准确率是0.93，意味着93%的预测是正确的。  

### 模型整体评估
accuracy：这表示整个模型的准确率，即模型正确分类的样本数占总样本数的比例。在这个例子中，准确率为0.93，意味着模型在所有预测中有93%是正确的。这是一个相对较高的准确率，表明模型整体性能很好。  

macro avg：这表示宏平均（Macro Average），它是对所有类别的精确度、召回率和F1分数的简单算术平均。它不考虑每个类别的样本数量，因此所有类别都被平等对待。这个例子中，宏平均的精确度、召回率和F1分数都是0.93，表明在平均意义上，模型对于所有类别的预测性能都很均衡。  

weighted avg：这表示加权平均（Weighted Average），它是对所有类别的精确度、召回率和F1分数的加权平均，其中权重是每个类别的支持（样本数）。这种方法考虑了类别不平衡的问题，更加关注样本数较多的类别。这个例子中，加权平均的精确度、召回率和F1分数也都是0.93，说明即使考虑了每个类别的样本数量，模型的整体性能也是相当高的。  

## 常见模型用法整理

1. 线性模型：线性回归&逻辑回归（一般用来二分类）

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression

# 线性回归示例
model = LinearRegression()
model.fit(X_train, y_train)

# 逻辑回归示例（分类）
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

2.支持向量机 SVM:  
分类用SVC  
回归用SVR  


from sklearn.svm import SVC, SVR

# 分类（SVC）
model = SVC()
model.fit(X_train, y_train)

# 回归（SVR）
model = SVR()
model.fit(X_train, y_train)


3. 集成方法
一般来讲效果最好  
为什么叫集成方法：随机森林是多个决策树的集合

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingRegressor

# 随机森林（分类）
model = RandomForestClassifier()
model.fit(X_train, y_train)

# 梯度提升（回归）
model = GradientBoostingRegressor()
model.fit(X_train, y_train)


GradientBoostingRegressor()

4. 聚类算法  
eg. k-means 

In [ ]:
from sklearn.cluster import KMeans

# K均值聚类
model = KMeans(n_clusters=3)
model.fit(X)


c:\Users\lianj\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1426: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=3)

5. 降维  
降低数据维度，用于数据可视化或改善模型性能。

In [ ]:
from sklearn.decomposition import PCA

# 主成分分析（PCA）
pca = PCA(n_components=2)
reduced_X = pca.fit_transform(X)


### 总结  
sklearn内置了许多机器学习的方法，需要大家选择合适的使用（一般先试试随机森林），  

机器学习中，数据是重要的，因此在上模型之前，数据预处理，特征工程的过程是必不可少的

### 课后作业
利用iris数据集或者python自带的其他数据集，利用课堂的知识进行特征工程，并且选择模型跑一下代码